In [9]:
#!pip install transformers
#!pip install huggingface_hub
#!pip install datasets

     -------------------------------------- 474.6/474.6 kB 6.0 MB/s eta 0:00:00
     ---------------------------------------- 132.9/132.9 kB ? eta 0:00:00
     --------------------------------------- 21.5/21.5 MB 19.9 MB/s eta 0:00:00
     ---------------------------------------- 323.6/323.6 kB ? eta 0:00:00
     ---------------------------------------- 61.7/61.7 kB ? eta 0:00:00
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.4
    Uninstalling dill-0.3.4:
      Successfully uninstalled dill-0.3.4


In [10]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset, load_metric

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [11]:
datasets = load_dataset("piqa")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16113 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3084 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1838 [00:00<?, ? examples/s]

Dataset piqa downloaded and prepared to C:/Users/shwet/.cache/huggingface/datasets/piqa/plain_text/1.1.0/6c611c1a9bf220943c4174e117d3b660859665baf1d43156230116185312d011. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

def to_df(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [16]:
show_random_elements(datasets["train"])

,goal,sol1,sol2,label
0,How to thicken tomato sauce without changing the flavor?,"Once the tomato sauce has come together, reduce it by increasing the heat and leaving the lid off the pot until it is about one fifth less","Before the tomato sauce has come together, reduce it by lower the heat and leaving the lid on the pot until it is about one fifth less",0
1,How to make Chocolate Mounds Candy bar Ice cream at home without an Ice cream Freezer.,"In a medium mixing bowl combine 4 cups chilled whipping cream, 14 oz. can sweetened condensed milk, and 1 16 ounce can chocolate flavored syrup (Like Hershey's). Beat with an electric mixer until soft Peaks form. Fold in 3/4 cup Mounds Candy Bars (coarsely chopped) and 1/4 cup shredded Coconut. Transfer mixture to 8x8x2 inch baking pan. Freeze about 8 hours until firm. Serve any time after 8 hours freeze.","In a medium mixing bowl combine 4 cups chilled whipping cream, 14 oz. can sweetened condensed milk, and 1 16 ounce can chocolate flavored syrup (Like Hershey's). Beat with an electric mixer until soft Peaks form. Fold in 3/4 cup Mounds Candy Bars (coarsely chopped) and 1/4 cup shredded Coconut Palm leaves. Transfer mixture to 8x8x2 inch baking pan. Freeze about 8 hours until firm. Serve any time after 8 hours freeze.",0
2,"To dissolve salt in water,",add the salt to room temperature water in a cup and stir gently with a spoon until dissolved.,add the salt to freezing water in a cup and stir gently with a spoon until dissolved.,0
3,Remove masking tape without leaving residue.,Warm first with lighter.,Warm first with blow dryer.,1
4,how do you remember someone who's died right after they die?,have a cremation for them.,have a memorial for them.,1
5,How can you improvise a water sprayer for your hose at home?,"Place a water or soda bottle on the end of your hose and duct tape it in place, then poke several holes in the bottle (strategically) to point and spray your hose.","Place a water or soda bottle on the shut off valve of your hose and duct tape it in place, then poke several holes in the bottle (strategically) to point and spray your hose.",0
6,To get router table to make curves on the corner of wood.,Use a corner radius router jig.,Use a rounded piece of wood as template.,0
7,How do I mop the floors?,Spread a little cleaner and water mixture onto the floor and mop it all around,Spread a little water and oil mixture into the floor and mop it all around,0
8,To attach the shelves to the back of the cabinet,Make sure the shelves have a place to rest on and use a soldering iron to attach everything,Make sure the shelves have a place to rest on and use a nail gun to attach everything,1
9,How do I spray with polyurethane?,Polyurethane comes in spray paint bottles. After purchasing a bottle shake it and spray it about a foot away from the surface and let it dry according to the instructions on the bottle before repeating.,Polyurethane comes in spray paint bottles. After purchasing a bottle shake it and spray it about 6 feet away from the surface and let it dry according to the instructions on the bottle before repeating.,0


In [20]:
datasets["train"][0]

{'goal': "When boiling butter, when it's ready, you can",
 'sol1': 'Pour it onto a plate',
 'sol2': 'Pour it into a jar',
 'label': 1}

## Preprocessing Data

In [17]:
from transformers import AutoTokenizer
model_checkpoint = "gpt2"
batch_size = 16   
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
from transformers import GPT2Tokenizer, GPT2Model, Trainer, TrainingArguments
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

model = GPT2Model.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)